In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n FT_DPO python=3.11
conda activate FT_DPO

In [ ]:
# INSTALA AS DEPENDÊNCIAS

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install transformers==4.42.4 datasets==2.19.1 accelerate==0.30.1 tokenizers==0.19.1 peft==0.10.0 bitsandbytes==0.43.1 trl==0.9.4

pip install ipykernel matplotlib
python -m ipykernel install --user --name=FT_DPO --display-name="FT_DPO"

In [ ]:
import tqdm as notebook_tqdm
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import DPOTrainer

In [1]:
# CONFIGURAÇÕES E DATASET

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "dataset_preferencias_juridico.jsonl" 
OUTPUT_DIR = "./modelo_juridico_dpo"

train_dataset = load_dataset('json', data_files={'train': DATASET_FILE}, split='train')
print("Dataset de preferências carregado:")
print(train_dataset)

In [2]:
# CARREGAMENTO DO MODELO E TOKENIZADOR
# >> Observe que neste exemplo estamos usando quantização para dar outro exemplo usando o ecossistema huggingface

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
# CONFIGURAÇÃO DO LoRA

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
)

In [ ]:
# CONFIGURAÇÃO E EXECUÇÃO DO TREINAMENTO DPO

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    bf16=True,
    overwrite_output_dir=True,
)

# Instanciar o DPOTrainer
# Espera as colunas 'prompt', 'chosen', e 'rejected' por padrão.
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1, # Parâmetro de regularização do DPO
)

print("\nIniciando o treinamento com DPO...")
dpo_trainer.train()

print(f"\nTreinamento concluído. Salvando o modelo em: {OUTPUT_DIR}")
dpo_trainer.save_model()